#### Structure du projet
Le projet consiste à :

**Extraire des données horaires** sur la qualité de l'air depuis l'API AirQuino.  
**Calculer la moyenne journalière** des niveaux de CO et PM2.5 pour chaque capteur.  
**Stocker ces données dans une base MongoDB**. 
**Fournir un dashboard superset** pour visualiser les données.  
**Prévoir les deux prochaines heures** des niveaux de CO et PM2.5 en utilisant un modèle ARIMA.  

##### Prérequis
Avant de pouvoir exécuter ce code, assurez-vous d'avoir installé les bibliothèques suivantes :

**requests** pour envoyer des requêtes HTTP.  
**pandas** pour manipuler les données.  
**pymongo** pour interagir avec la base de données MongoDB.  
**statsmodels** pour les prévisions avec ARIMA.  
Si ces packages ne sont pas installés, vous pouvez les installer en utilisant pip :    
**pip install requests pandas pymongo statsmodels**  

##### Installation de MongoDB
MongoDB doit être installé et en cours d'exécution localement pour stocker les moyennes journalières. Vous pouvez télécharger MongoDB depuis ici https://www.mongodb.com/try/download/community.

**Installez MongoDB**.
Assurez-vous que le serveur MongoDB est en cours d'exécution :

In [2]:
# Importer le package requests
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

## Extraction de données

**Envoyer une requête à l'API AirQuino**  
Le script commence par envoyer une requête à l'API pour récupérer des données horaires de la station.  
Si la réponse est correcte (statut 200), les données JSON sont extraites et traitées.

In [3]:
# l'URL de l'API 
url = r'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283164601'

# Effectuer la requête GET
response = requests.get(url) 

# Vérifier si la réponse est réussi 
if response.status_code == 200:
    # Traiter les données JSON
    data = response.json()
else:
    print(f"Erreur : {response.status_code}")

**Conversion en DataFrame**  
Les données récupérées sont transformées en DataFrame pour une manipulation facile.

In [4]:
# convertion des données en dataFrame 
df = pd.DataFrame(data['data'])

# Affichage du tableau 
df.head()

,timestamp,CO,T,T. int.,NO2,O3,PM10,PM2.5,RH
0,2023-09-15 20:00:00,0.0,26.223333,367.63333,19.783640,24.070265,14.866667,5.000000,90.67667
1,2023-09-15 21:00:00,0.0,26.186666,366.93332,19.460062,25.548030,16.000000,5.100000,91.33667
2,2023-09-15 22:00:00,0.0,26.210001,367.23334,19.419436,25.756544,15.133333,4.033333,91.37667
3,2023-09-15 23:00:00,0.0,26.013334,365.23334,19.018366,27.521160,13.833333,3.600000,91.71333
4,2023-09-16 00:00:00,0.0,25.723333,362.13333,18.552538,29.524677,11.200000,2.066667,93.78000


**Transformation des données**  

La colonne timestamp est convertie en format datetime.  
La date est extraite pour faciliter les calculs journaliers.

In [ ]:
# Convertir le timestamp en format datetime 
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
# Extraire la date seulement
df['date'] = df['timestamp'].dt.date

**Calcul des moyennes journalières**  
Les moyennes des niveaux de CO et PM2.5 sont calculées par jour et pour chaque capteur.

In [ ]:
# Calculer la moyenne par jour pour chaque capteur 
daily_avg = df.groupby(['date'])[['CO','PM2.5']].mean().reset_index()

daily_avg.head()

**Connexion à MongoDB**   
Le script se connecte à une base de données MongoDB locale.

In [ ]:
# Connexion a MongoDB 
client = MongoClient('mongodb://localhost:27017/') 
db = client['data354']
collection = db['daily_averages']

**Insertion des données dans MongoDB**  
Les moyennes journalières sont formatées et insérées dans la collection MongoDB.

In [ ]:
# Préparer les données a insérer dans MongoDB 
records = []
for _,row in daily_avg.iterrows():
    record = {
        "Station_name": data['header']['station_name'], 
        "date": row['date'].strftime('%Y-%m-%d'), 
        "CO_avg": row['CO'], 
        "PM2.5_avg": row['PM2.5'],
        "station_lat": data['header']['station_lat'],
        "station_lon": data['header']['station_lon']
    }
    records.append(record)

# Insérer les données dans la collection MongoDB 
collection.insert_many(records)

print("Données insérées avec succès dans MongoDB!")

In [ ]:
# daily_avg.to_csv('challenge354.csv', index=False)

In [ ]:
# df.to_csv('data.csv', index=False)

**Prévision avec ARIMA**  
Le modèle ARIMA est utilisé pour prévoir les niveaux de CO et PM2.5 pour les deux prochaines heures.

In [ ]:
# Convertir la colonne timestamp en datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir le timestamp comme index
df.set_index('timestamp', inplace=True)

# Définir explicitement la fréquence à 'h' pour les données horaires
df = df.asfreq('h')

# Appliquer ARIMA pour prédire CO
model_co = ARIMA(df['CO'], order=(5, 1, 0))  
model_co_fit = model_co.fit()

# Appliquer ARIMA pour prédire PM2.5
model_pm25 = ARIMA(df['PM2.5'], order=(5, 1, 0)) 
model_pm25_fit = model_pm25.fit()

# Faire des prévisions pour les 2 prochaines heures pour CO
forecast_co = model_co_fit.forecast(steps=2)
print("Prévisions ARIMA pour CO:", forecast_co, "\n")

# Faire des prévisions pour les 2 prochaines heures pour PM2.5
forecast_pm25 = model_pm25_fit.forecast(steps=2)
print("Prévisions ARIMA pour PM2.5:", forecast_pm25)